In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image
import os

In [76]:
path = "/Users/fareskallel/Desktop/Project_OD/maps"
maps = [m for m in os.listdir(path)]
pixel_arrays = []
for i,m in enumerate(maps):
    try:
        path_map = os.path.join(path,f"map{i+1}/map{i+1}.png")
        im = Image.open(path_map)
        width, height = im.size
        pixel_values = list(im.getdata())
        pixel_values = np.array(pixel_values)
        pixel_values = pixel_values.reshape((width, height, 3))
        pixel_arrays.append(pixel_values)
    except: break
    
print(f"{len(pixel_arrays)} maps were loaded")

2260 maps were loaded


In [82]:
pixels = [np.float32(p/255) for p in pixel_arrays]
train_data = pixels[int(len(pixels)*0.2):]
test_data = pixels[:int(len(pixels)*0.2)] 
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, num_workers=0,drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, num_workers=0,drop_last=True)


In [77]:
#Define the Convolutional Autoencoder
import torchvision
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
       
        #Encoder
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
       
        #Decoder
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(16, 3, 2, stride=2,output_padding = 1)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.t_conv1(x))
        x = torch.sigmoid(self.t_conv2(x))
        
        return x

In [78]:
#Instantiate the model
model = ConvAutoencoder()
print(model)

ConvAutoencoder(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (t_conv1): ConvTranspose2d(4, 16, kernel_size=(2, 2), stride=(2, 2))
  (t_conv2): ConvTranspose2d(16, 3, kernel_size=(2, 2), stride=(2, 2), output_padding=(1, 1))
)


In [79]:
device = get_device()
print(device)
model.to(device)

cpu


ConvAutoencoder(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (t_conv1): ConvTranspose2d(4, 16, kernel_size=(2, 2), stride=(2, 2))
  (t_conv2): ConvTranspose2d(16, 3, kernel_size=(2, 2), stride=(2, 2), output_padding=(1, 1))
)

In [84]:
#Epochs
# only difference is CUDA! we run on cpu and he on gpu.

#Loss function
criterion = nn.BCELoss()

#Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

n_epochs = 10
train_loss = []
for epoch in range(1, n_epochs+1):
    for data in train_loader:
        batch_loss = []
        images = data.view((32,3,101,101))
#         print(images.shape)
        images = images.to(device)
        optimizer.zero_grad()
        outputs = model(images)
#         print(outputs.shape)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())
    epoch_loss = np.mean(batch_loss)
#     epoch_loss += loss.item()*images.size(0)
#         print(loss.item()*images.size(0))
          
#     train_loss = train_loss/len(loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, epoch_loss))

Epoch: 1 	Training Loss: 0.653256
Epoch: 2 	Training Loss: 0.650545
Epoch: 3 	Training Loss: 0.650242
Epoch: 4 	Training Loss: 0.650097
Epoch: 5 	Training Loss: 0.650031
Epoch: 6 	Training Loss: 0.649995
Epoch: 7 	Training Loss: 0.649971
Epoch: 8 	Training Loss: 0.649955
Epoch: 9 	Training Loss: 0.649944
Epoch: 10 	Training Loss: 0.649935


In [83]:
dataiter = iter(loader)
notro_images = dataiter.next()

In [88]:
notro_images = notro_images.view((32,3,101,101))
images.shape

torch.Size([32, 3, 101, 101])

In [8]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device

In [31]:
def imshow(img):
    img = img / 2 + 0.5  
    plt.imshow(np.transpose(img, (1, 2, 0))) 

In [92]:
#Batch of test images
dataiter = iter(test_loader)
images = dataiter.next()
print(images.shape)
#Sample outputs
output = model(images.view((32,3,101,101)))
# images = images.numpy()

output = output.view(32, 3, 101, 101)
output = output.detach().numpy()

reconst_input = Image.fromarray(np.array((images[31].reshape((101,101,3)))*255).astype('uint8'))
reconst_output = Image.fromarray(np.array((output[31].reshape((101,101,3)))*255).astype('uint8'))

reconst_input.show()
reconst_output.show()

# data = np.asarray(images[14])
# data = np.reshape(data, (101, 101, 3))
# img1 = Image.fromarray(data, 'RGB')
# data = np.asarray(output[14])
# data = np.reshape(data, (101, 101, 3))
# img = Image.fromarray(data, 'RGB')

# #Original Images
# print("Original Images")
# fig, axes = plt.subplots(nrows=1, ncols=5, sharex=True, sharey=True, figsize=(12,4))
# for idx in np.arange(5):
#     ax = fig.add_subplot(1, 5, idx+1, xticks=[], yticks=[])
#     imshow(images[idx])
#     ax.set_title(classes[labels[idx]])
# plt.show()

# #Reconstructed Images
# print('Reconstructed Images')
# fig, axes = plt.subplots(nrows=1, ncols=5, sharex=True, sharey=True, figsize=(12,4))
# for idx in np.arange(5):
#     ax = fig.add_subplot(1, 5, idx+1, xticks=[], yticks=[])
#     imshow(output[idx])
#     ax.set_title(classes[labels[idx]])
# plt.show() 

torch.Size([32, 101, 101, 3])


In [ ]:
im = Image.fromarray(my_array.reshape((50,50)).astype('uint8')*255)

In [75]:
path_map = "/Users/fareskallel/Desktop/Project_OD/maps/map1/map1.png"
im = Image.open(path_map)
im.show()
width, height = im.size
pixel_values = list(im.getdata())
pixel_values = np.array(pixel_values)
pixel_values = pixel_values.reshape((width, height, 3))

reconstruction = Image.fromarray(pixel_values.reshape((101,101,3)).astype('uint8'))
reconstruction.show()